# Compute DWPCs for all metapaths

In [1]:
import numpy as np
import pandas as pd
import pandas
import scipy.sparse
from scipy import sparse
import itertools
import random
import time
import tqdm

from hetmech.degree_weight import dwpc
from hetmech.matrix import metaedge_to_adjacency_matrix
import hetmech.degree_group
import hetmech.degree_weight
from hetmech.hetmat import HetMat

In [2]:
hetmat = HetMat('../data/hetionet-v1.0.hetmat/')
hetmech.degree_weight.default_dwwc_method = hetmech.degree_weight.dwwc_recursive

All metapaths up to length 3 saving PC and DWPC matrices on disk -- actual hetnet.

In [3]:
metapaths = hetmat.metagraph.extract_all_metapaths(3, exclude_inverts=True)
# random.seed(1)
# metapaths = random.sample(metapaths, k=2)
len(metapaths)

2

### Compute path counts

In [4]:
%%time
hetmat.path_counts_cache = hetmech.hetmat.PathCountPriorityCache(hetmat, allocate_GB=16)
for metapath in tqdm.tqdm(metapaths):
    row, col, dwpc_matrix = dwpc(hetmat, metapath, damping=0, dense_threshold=1, dtype='uint64')
    path = hetmat.get_path_counts_path(metapath, 'dwpc', 0.0, None)
    hetmech.hetmat.save_matrix(dwpc_matrix, path)
    del dwpc_matrix

100%|██████████| 2/2 [00:00<00:00,  3.00it/s]

CPU times: user 643 ms, sys: 22.4 ms, total: 666 ms
Wall time: 674 ms


### Compute DWPCs

In [5]:
%%time
hetmat.path_counts_cache = hetmech.hetmat.PathCountPriorityCache(hetmat, allocate_GB=16)
mean_dwpcs = dict()
for metapath in tqdm.tqdm(metapaths):
    # Compute degree-weighted path counts
    row, col, dwpc_matrix = dwpc(hetmat, metapath, damping=0.5, dense_threshold=1)
    mean_dwpcs[(metapath, 'dwpc', 0.5)] = dwpc_matrix.mean()
    path = hetmat.get_path_counts_path(metapath, 'dwpc', 0.5, None)
    hetmech.hetmat.save_matrix(dwpc_matrix, path)
    del dwpc_matrix

100%|██████████| 2/2 [00:00<00:00,  2.94it/s]

CPU times: user 670 ms, sys: 15.8 ms, total: 686 ms
Wall time: 685 ms


In [6]:
hetmat.path_counts_cache = None

Read in matrices to get scaling factor (mean). Then compute all DGP for all permutations.

In [8]:
for name, permat in hetmat.permutations.items():
    permat.path_counts_cache = hetmech.hetmat.PathCountPriorityCache(permat, allocate_GB=16)
    for metapath in metapaths:
        _, _, matrix = dwpc(permat, metapath, damping=0.5, dense_threshold=0.7)
        source_deg_to_ind, target_deg_to_ind = hetmech.degree_group.metapath_to_degree_dicts(permat, metapath)
        scaler = mean_dwpcs[(metapath, 'dwpc', 0.5)]
        row_generator = hetmech.degree_group.generate_degree_group_stats(
            source_deg_to_ind, target_deg_to_ind, matrix, scale=True, scaler=scaler)
        degree_stats_df = (
            pd.DataFrame(row_generator)
            .set_index(['source_degree', 'target_degree'])
        )
        path = permat.directory.joinpath('degree-grouped-path-counts', 'dwpc-0.5', f'{metapath}.tsv')
        path.parent.mkdir(parents=True, exist_ok=True)
        degree_stats_df.to_csv(path, sep='\t')
    permat.path_counts_cache = None

Create multi-permutation DGP summary metrics.

In [12]:
for metapath in metapaths:
    degree_stats_df = None
    for permat in hetmat.permutations.values():
        path = permat.directory.joinpath('degree-grouped-path-counts', 'dwpc-0.5', f'{metapath}.tsv')
        df = (
            pandas.read_table(path)
            .set_index(['source_degree', 'target_degree'])
            .assign(n_perms=1)
        )
        if degree_stats_df is None:
            degree_stats_df = df
        else:
            degree_stats_df += df
    degree_stats_df = hetmech.degree_group.compute_summary_metrics(degree_stats_df)
    degree_stats_df.drop(columns=['sum', 'sum_of_squares'], inplace=True)
    path = hetmat.directory.joinpath('adjusted-path-counts', 'dwpc-0.5', 'degree-grouped-permutations', f'{metapath}.tsv')
    path.parent.mkdir(parents=True, exist_ok=True)
    degree_stats_df.to_csv(path, sep='\t')